In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
import os
from torch import nn
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# loop through all files in "data/good_data" and concatenta them into one dataframe
df = pd.concat([pd.read_csv(f"data/good_data/{file}") for file in os.listdir("data/good_data")])
# keep only common_name, condition, latitude_coordinate, longitude_coordinate, and native columns
df = df[['common_name', 'condition', 'latitude_coordinate', 'longitude_coordinate', 'native']]
# convert condition to numerical
df['condition'] = df['condition'].replace({'excellent': 4, 'good': 3, 'fair': 2, 'poor': 1, 'dead/dying': 0, 'dead': 0})
# one hot "common_name" column
df = pd.get_dummies(df, columns=["common_name"])
# one hot native column
df = pd.get_dummies(df, columns=["native"])
# drop native_no_info column
df.drop(columns=['native_no_info'], inplace=True)
# drop rows where condition is null
df = df.dropna(subset=['condition'])


# split into X and y
X = df.drop('condition', axis=1)
y = df['condition']

# convert from boolean to int
y = y.astype(int)
X = X.astype(float)

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [4]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [5]:

# convert data to PyTorch tensors
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()
y_train = torch.from_numpy(y_train.values).long()
y_test = torch.from_numpy(y_test.values).long()

# parameters
input_size = len(X.columns)
num_classes = len(df['condition'].unique())
num_epochs = 100

In [6]:
# grid search for hyperparameters
for learning_rate in [0.0001, 0.001, 0.01, 0.1]:
    for batch_size in [100, 200, 300, 400, 500]:
        for hidden_size in [500, 1000, 1500]:
            # create model
            model = NeuralNet(input_size, hidden_size, num_classes)

            # loss and optimizer
            criterion = nn.CrossEntropyLoss()
            # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

            # train model
            total_step = len(X_train)
            loss_list = []
            acc_list = []
            for epoch in range(num_epochs):
                for i in range(0, total_step, batch_size):
                    # get batch
                    X_batch = X_train[i:i+batch_size]
                    y_batch = y_train[i:i+batch_size]
                    
                    # forward pass
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    loss_list.append(loss.item())
                    
                    # backward and optimize
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    # accuracy
                    total = y_batch.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    correct = (predicted == y_batch).sum().item()
                    acc_list.append(correct / total)
                    
                if (epoch+1) % 10 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {correct / total:.4f}')
            print(f'learning_rate: {learning_rate}, batch_size: {batch_size}, hidden_size: {hidden_size}, accuracy: {correct / total:.4f}')

# test model
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(len(X_test)):
        X = X_test[i].unsqueeze(0)
        y = y_test[i]
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        n_samples += y.size(0)
        n_correct += (predicted == y).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test images: {acc} %')

# save model
torch.save(model.state_dict(), 'model.ckpt')

Epoch [10/100], Loss: 1.0767, Accuracy: 0.4494
Epoch [20/100], Loss: 1.0773, Accuracy: 0.4494
Epoch [30/100], Loss: 1.0778, Accuracy: 0.4494
Epoch [40/100], Loss: 1.0781, Accuracy: 0.4494
Epoch [50/100], Loss: 1.0783, Accuracy: 0.4494
Epoch [60/100], Loss: 1.0785, Accuracy: 0.4494
Epoch [70/100], Loss: 1.0786, Accuracy: 0.4494
Epoch [80/100], Loss: 1.0787, Accuracy: 0.4494
Epoch [90/100], Loss: 1.0787, Accuracy: 0.4494
Epoch [100/100], Loss: 1.0787, Accuracy: 0.4494
learning_rate: 0.0001, batch_size: 100, hidden_size: 500, accuracy: 0.4494
Epoch [10/100], Loss: 1.0806, Accuracy: 0.4494
Epoch [20/100], Loss: 1.0813, Accuracy: 0.4494
Epoch [30/100], Loss: 1.0814, Accuracy: 0.4494
Epoch [40/100], Loss: 1.0814, Accuracy: 0.4494
Epoch [50/100], Loss: 1.0813, Accuracy: 0.4494


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i in range(len(X_test)):
        X = X_test[i].unsqueeze(0)
        y = y_test[i]
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        n_samples += y.size(0)
        n_correct += (predicted == y).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test data: {acc} %')

In [1]:
# plot accuracy and loss
plt.plot(loss_list)
plt.title('Loss')
plt.show()
plt.plot(acc_list)
plt.title('Accuracy')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# return optimal parameters from training
